In [16]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import uuid
import string
import re

In [23]:
data = pd.read_csv('./data/data.csv')
data

,Unnamed: 0,id,title,distributionDate,author,text
0,0,19930007195,Experiments on a round turbulent buoyant plume,2019-06-26T00:00:00.0000000+00:00,"Shabbir, Aamir",b'NASA\n\nTechnical\n\nMemorandum\n\nICOMP-92-...
1,1,19810022682,"Terminal area automatic navigation, guidance, ...",2019-06-21T00:00:00.0000000+00:00,"Pines, S.",b'NASA\nCR\n345 l-\n\npt.1\n\nNASA Contractor ...
2,2,19870017020,Opportunities for PV applications,2013-08-29T00:00:00.0000000+00:00,"Scott-Monck, John A.","b""N87-26453\nOPPORIUNITIES\n\n3ohn\n\nA.\n\nFO..."
3,3,19770068911,Brake friction materials: A market survey,2019-06-20T00:00:00.0000000+00:00,"Wilhem, J. P.",b'BRAKE FRICTION\n\nMATERIALS:\n\nA MARKET SUR...
4,4,19660015708,Thermogravimetric instrument,2019-05-28T00:00:00.0000000+00:00,"Stravs, S.",b'i\n\n!\n\n.\n\n\'\n\nd F\n/?-\n\n5-6\n\nTHER...
...,...,...,...,...,...,...
3098,3098,20040040154,Synthetic Vision Enhances Situation Awareness ...,2019-07-12T00:00:00.0000000+00:00,"Kramer, Lynda J.","b'AIAA\'s 3rd Annual Aviation Technology, Inte..."
3099,3099,19710000275,A real-time statistical time-series analyzer,2019-06-17T00:00:00.0000000+00:00,"Stewart, C. H.","b""August 1971\n\n\t\n\nBrief 71-10276\n\nNASA ..."
3100,3100,20070038208,The GeoEye Satellite Constellation,2018-06-11T00:00:00.0000000+00:00,"Dial, Gene",b'The GeoEye Satellite\nConstellation\n\nGene ...
3101,3101,19890005791,Electrodynamic tether system study: Extended s...,2019-06-24T00:00:00.0000000+00:00,NaN,b'Electrodynamic Tether System Study\n/Extende...


In [24]:
# take only date of day
data['distributionDate'] = data['distributionDate'].str.split(' ', expand = True)[0]
# delete index column
data = data.drop('Unnamed: 0', 1)

C:\Users\motaz\AppData\Local\Temp\ipykernel_20268\1708228254.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('Unnamed: 0', 1)


In [25]:
data

,id,title,distributionDate,author,text
0,19930007195,Experiments on a round turbulent buoyant plume,2019-06-26T00:00:00.0000000+00:00,"Shabbir, Aamir",b'NASA\n\nTechnical\n\nMemorandum\n\nICOMP-92-...
1,19810022682,"Terminal area automatic navigation, guidance, ...",2019-06-21T00:00:00.0000000+00:00,"Pines, S.",b'NASA\nCR\n345 l-\n\npt.1\n\nNASA Contractor ...
2,19870017020,Opportunities for PV applications,2013-08-29T00:00:00.0000000+00:00,"Scott-Monck, John A.","b""N87-26453\nOPPORIUNITIES\n\n3ohn\n\nA.\n\nFO..."
3,19770068911,Brake friction materials: A market survey,2019-06-20T00:00:00.0000000+00:00,"Wilhem, J. P.",b'BRAKE FRICTION\n\nMATERIALS:\n\nA MARKET SUR...
4,19660015708,Thermogravimetric instrument,2019-05-28T00:00:00.0000000+00:00,"Stravs, S.",b'i\n\n!\n\n.\n\n\'\n\nd F\n/?-\n\n5-6\n\nTHER...
...,...,...,...,...,...
3098,20040040154,Synthetic Vision Enhances Situation Awareness ...,2019-07-12T00:00:00.0000000+00:00,"Kramer, Lynda J.","b'AIAA\'s 3rd Annual Aviation Technology, Inte..."
3099,19710000275,A real-time statistical time-series analyzer,2019-06-17T00:00:00.0000000+00:00,"Stewart, C. H.","b""August 1971\n\n\t\n\nBrief 71-10276\n\nNASA ..."
3100,20070038208,The GeoEye Satellite Constellation,2018-06-11T00:00:00.0000000+00:00,"Dial, Gene",b'The GeoEye Satellite\nConstellation\n\nGene ...
3101,19890005791,Electrodynamic tether system study: Extended s...,2019-06-24T00:00:00.0000000+00:00,NaN,b'Electrodynamic Tether System Study\n/Extende...


In [ ]:
es = Elasticsearch()
index_name = 'NTRS'

In [ ]:
# create an index with custom mapping
es.indices.create(index=index_name, ignore=400, body={
    'mappings': {
        'dynamic': 'strict',
        'properties':{
            'id': {'type': 'integer'},
            'title': {'type': 'text'},
            'distributionDate': {'type': 'date'},
            'author': {'type': 'text'},
            'text': {
                'type': 'text',
                "fielddata": True
            }
        }
    }
})

In [47]:
data_list = data.values.tolist()

In [59]:
actions = []

for record in range(len(data_list)):
    
    if record % 1000 != 0:
        actions.append({
            '_index': index_name,
            '_type': '_doc',
            '_id': uuid.uuid4().int,
            '_source': {
                "id": data_list[record][0],
                "title": data_list[record][1],
                "distributionDate": data_list[record][2],
                "author": data_list[record][3],
                "text": data_list[record][4]
                }
        })
    else:
        helpers.bulk(es, actions)
        

# inverted index are Done